In [1]:
# Formação Cientista de Dados - Fernando Amaral e Jones Granatyr
# Colaboração: Adriano Santos

# Algoritmo ECLAT

In [100]:
# Este código Python importa diversas bibliotecas essenciais para operações matemáticas, manipulação de datas e processamento de argumentos de linha de comando. As bibliotecas incluem 'os' e 'sys' para operações do sistema, 'numpy' para cálculos numéricos, 'datetime' para manipulação de datas e horas, 'numpy.linalg' para álgebra linear e 'optparse' e 'argparse' para análise de argumentos de linha de comando. Essas importações fornecem as ferramentas necessárias para realizar uma variedade de tarefas, desde cálculos matemáticos avançados até a gestão de argumentos passados ao script.

import os;
import sys;
import numpy as np;
import datetime as dt;
from numpy import linalg as LA;
import optparse;
import argparse;
import pandas as pd
from pyECLAT import ECLAT

# %pip install pyECLAT

In [102]:
# Variável global
# Contendo todos os padrões frequentes com suas tid's
F = []

# Cálculo de tempo
start_time = 0
end_time = 0

# Classe Pattern (Padrão):
class Pattern:
    def __init__(self, item, tids):
        # Lista de IDs de itens no padrão
        self.item_id = item
        # tid do padrão
        self.tid_list = tids

    # Operação de união dos IDs de itens para dois padrões para obter o ID do padrão candidato
    def union_id(self, next_node):
        t = set(self.item_id)
        tt = set(next_node.item_id)
        new = t | tt
        new_id = list(new)
        new_id.sort()
        return new_id

    # Interseção das tid's de dois padrões para encontrar a tid do padrão candidato
    def intersec_tid_list(self, next_node):
        t = set(self.tid_list)
        tt = set(next_node.tid_list)
        new = t & tt
        new_list = list(new)
        new_list.sort()
        return new_list

    # Retorna o valor de suporte para o padrão
    def getSup(self):
        return len(self.tid_list)

    # Geração de candidatos e verificação de suporte
    def generate_check(self, n, minsup):
        # Gerando a lista de tid primeiro
        temp_tid_list = self.intersec_tid_list(n)

        # Se o suporte para o novo candidato for >= minsup, então gere o ID para esse candidato
        if len(temp_tid_list) >= minsup:
            temp_id = self.union_id(n)
            return (temp_id, temp_tid_list)
        else:
            return ([], [])

    def pattern_print(self):
        pattern_ID = str("")
        for i in self.item_id:
            pattern_ID = pattern_ID + str(i) + ' '
        print(pattern_ID + '\t\t : ' + str(self.tid_list))

# Fim da classe Pattern (Padrão)


In [103]:
# Classe Pattern Store:
class PatternStore:
    def __init__(self):
        self.Pattern_list = []

    # Adiciona um grupo inteiro de padrões à lista
    def addGroup(self, list_nodes):
        for n in list_nodes:
            self.Pattern_list.append(n)

    # Retorna o i-ésimo padrão da lista, se disponível
    def getNode(self, index):
        if index < len(self.Pattern_list):
            return self.Pattern_list[index]
        else:
            return []

    # Para seguir o método DFS: Função recursiva é usada - para calcular todos os padrões frequentes da lista de padrões frequentes do 1º nível
    def Eclat(self, minsup):
        for node in self.Pattern_list:
            F.append(node)
            new_P = PatternStore()
            # Como todos os padrões são ordenados inicialmente, obtenha apenas o próximo padrão do atual
            index = self.Pattern_list.index(node)
            i = index + 1
            n = self.getNode(i)

            while n:
                # Aqui estou realizando 2 etapas juntas: 1- geração de candidatos e 2- verificação de minsup
                (temp_id, temp_tid_list) = node.generate_check(n, minsup)
                # se temp_id não estiver vazio
                if temp_id:
                    new_P.Pattern_list.append(Pattern(temp_id, temp_tid_list))
                i = i + 1
                # obtenha o próximo nó da lista
                n = self.getNode(i)
            # se houver algum possível padrão filho, vá para o filho primeiro, DFS
            if new_P.Pattern_list:
                new_P.Eclat(minsup)

# Fim da classe PatternStore


In [105]:
class DBReader:

    def __init__(self, arquivo_para_ler):
        self.file_id = arquivo_para_ler

    def lerArquivo(self):
        # Armazena cada transação
        self.transacoes = []
        # Armazena o número de itens em cada transação
        self.num_itens = []

        for linha in self.file_id:
            tupla = linha.split(" ")
            minha_lista = []
            self.num_itens.append(tupla[0])

            for posicao in range(1, len(tupla)):
                minha_lista.append(tupla[posicao])

            self.transacoes.append(minha_lista)

    # A partir das transações: gera padrões e prepara uma lista
    def gerarListaPadroes(self):
        # Todos os itens
        temp = [item for sublist in self.transacoes for item in sublist]

        # Removendo duplicatas
        meu_conjunto = set(temp)
        self.itens = list(meu_conjunto)

        # Colocando em ordem ordenada
        self.itens.sort()

        # Gerar lista de padrões para o nível 1
        self.nivel_1 = []

        # Iniciar o cronômetro, pois a geração de candidatos para o nível 1 faz parte do algoritmo Eclat.
        start_time = dt.datetime.now()

        # Gerar todos os candidatos para o nível 1
        for item in self.itens:
            item_lista = []
            item_lista.append(item)
            self.nivel_1.append(Padrao(item_lista, []))

        # Gerar lista de tid para cada item no nível 1
        i = 0
        for transacao in self.transacoes:
            i = i + 1
            transacao.sort()
            for id_item in transacao:
                indice = self.itens.index(id_item)
                self.nivel_1[indice].lista_tid.append(i)

        return start_time

    # Retorna padrões frequentes
    def obterPadroesFrequentes(self, minsup):
        self.P = []
        for n in self.nivel_1:
            c = int(n.obterSup())
            if c >= minsup:
                # print c;
                self.P.append(n)
        return self.P

# Fim da classe DBReader


In [123]:
import argparse
import os
import sys
import datetime as dt

def main(file, support):
	# Definindo a flag de impressão
	p_flag = 1

	# Configurando o parser de argumentos
	parser = argparse.ArgumentParser()
	parser.add_argument("-f", "--filename")
	parser.add_argument("-s", "--supnum", type=int)
	parser.add_argument("-p", "--print_flag", action="store_true")
	args = parser.parse_args()
	
	try:	
		# Nome do arquivo
		f_name = file
		fileToRead = open(f_name)
		
		# Valor mínimo de suporte
		minsup = support
		
		# Configurando a flag de impressão com base nos argumentos
		if args.print_flag:
			p_flag = 1;	
	except(IOError, IndexError):
		print('Nome de arquivo inválido'+os.linesep)
		sys.exit(1)

	print('minsup = '+str(minsup))

	# Criando um objeto DBReader
	reader = DBReader(fileToRead)
	
	# Lendo o arquivo
	reader.readFile()
	
	# Geração de padrões para o nível 1 - Ele retornará o horário de início da geração de candidatos.
	start_time = reader.genPatternList()
	
	# Obtendo padrões frequentes de todos os candidatos do nível 1:
	level1_P = reader.getFrequent(minsup)

	# Criando um objeto Pattern Store
	PttStr = PatternStore()
	
	# Adicionando padrões frequentes do nível 1
	PttStr.addGroup(level1_P)

	# Algoritmo Eclat
	PttStr.Eclat(minsup)
	
	end_time = dt.datetime.now()
	
	# Calculando o tempo de computação
	time = end_time - start_time
	
	print('Tempo de computação = '+ str(time.total_seconds()) + ' segundos')
	
	if p_flag == 1:
		print('Padrão\t\t : Lista_de_TID');	
		for n in F:
			n.pattern_print()

# Fim da função principal


In [125]:
# Geração dos itens frequentes

if	__name__== "__main__":
	main('../dados/transacoes2.txt', 2)
 

usage: ipykernel_launcher.py [-h] [-f FILENAME] [-s SUPNUM] [-p]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9028 --control=9026 --hb=9025 --Session.signature_scheme="hmac-sha256" --Session.key=b"cfe8c060-52e2-430e-8824-0adc0e7083e2" --shell=9027 --transport="tcp" --iopub=9029


SystemExit: 2

In [122]:
# Importar a biblioteca pandas
import pandas as pd

# Carregar os dados do arquivo CSV
dados = pd.read_csv('../dados/transacoes2.txt', header=None)

# Criar uma instância do ECLAT com os dados carregados
eclat_instance = ECLAT(data=dados, verbose=True)

# Acessar o DataFrame binário gerado pelo ECLAT
eclat_instance.df_bin

# Acessar os itens únicos presentes nos dados
eclat_instance.uniq_

# Executar o ECLAT com os parâmetros desejados
get_ECLAT_indexes, get_ECLAT_supports = eclat_instance.fit(
    min_support=0.08,
    min_combination=1,
    max_combination=3,
    separator=' & ',
    verbose=True
)

# Obter os índices das combinações frequentes encontradas
get_ECLAT_indexes

# Obter os suportes das combinações frequentes encontradas
get_ECLAT_supports


100%|██████████| 6/6 [00:00<00:00, 1000.79it/s]


Combination 1 by 1


6it [00:00, 96.83it/s]


Combination 2 by 2


15it [00:00, 130.51it/s]


Combination 3 by 3


20it [00:00, 102.10it/s]


{'4 Cerveja Carvao Pao Maionese': 0.16666666666666666,
 '1 Cerveja': 0.16666666666666666,
 '2 Carvao Pao': 0.16666666666666666,
 '3 Cerveja Carvao Pao': 0.16666666666666666,
 '1 Carvao': 0.16666666666666666,
 '3 Cerveja Carvao Maionese': 0.16666666666666666}